# Khit Thit Webscraper (1): Scraping Headers, Dates, and Contents of Multiple Pages Without Duplicates
by <a href="https://www.linkedin.com/in/la-wun-nannda-b047681b5/">`La Wun Nannda`</a>

## Libraries

In [1]:
# import libraries
from bs4 import BeautifulSoup # this module helps in web scrapping
import requests  # this module helps us to download a web page
import pandas as pd

## Functions for Scraping A Single Page

In [2]:
# function to get soup with URL input
def web_scraper(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html5lib')
    return soup

In [3]:
# function to get specific elements within a soup
def soup_parser(soup):
    news_headers_soup = soup.find_all("a", {"class":None, "rel":"bookmark"})
    datetime_soup = soup.find_all("time")
    return news_headers_soup, datetime_soup

In [4]:
# function to extract Burmese content from a content url
def content_scraper(content_url, soup):
    burmese_content = ""
    alphabets = ['a', 'b', 'c', 'd', 'e',
                'f', 'g', 'h', 'i', 'j',
                'k', 'l', 'm', 'n', 'o',
                'p', 'q', 'r', 's', 't',
                'u', 'v', 'w', 'x', 'y',
                'z']
    
    symbols = ["\'", "(", ")"]
    
    for p_element in soup.find_all("p"):
        try: # None Type can cause error
            content = p_element.string.strip()
            for char in content:
                if (char.lower() in alphabets) or (char in symbols): # do not add non-Burmese characters or symbols
                    continue
                burmese_content += char # add Burmese characters only
        except:
            pass
    return burmese_content

In [5]:
# function to create lists for a page
'''Lists are to be appended to original ones'''
def list_append_per_page(news_headers_soup, datetime_soup):
    news_headers_per_page = []
    datetime_per_page = []
    contents_per_page = []
    
    if len(news_headers_soup)==len(datetime_soup): # each header should have a corresponding date
        
        for i in range(len(news_headers_soup)): # get index of headers for one page
    
            # list 1 for multiple headers in a page
            try: # for news headers without video tag # video tagged ones will cause errors
                news_headers_per_page.append(news_headers_soup[i].string.strip()) # convert 'BeautifulSoup string' to 'Python string' # add content to list 1
            except AttributeError: # # for news headers with video tag
                '''list() is used to convert 'BeautifulSoup tag' object to 'list' to enable iteration'''
                news_headers_per_page.append(list(news_headers_soup[i].span)[1].strip()) # convert 'BeautifulSoup string' to 'Python string' # add content to list 1
    
            # list 2 for date and time in a page
            datetime_per_page.append(datetime_soup[i].string.strip()) # convert 'BeautifulSoup string' to 'Python string' # add content to list 2

            # list 3 for contents of all headers in a page (contents of multiple headers)
            content_url = news_headers_soup[i].attrs['href'] # get a link from 'n' element
            content_soup = web_scraper(content_url) # pass the link to create a new soup
            content_per_header = content_scraper(content_url, content_soup) # this new soup is used for content scraping
            contents_per_page.append(content_per_header)
        
        if (len(news_headers_per_page)==len(news_headers_soup)) & (len(datetime_per_page)==len(datetime_soup)) & (len(contents_per_page)!=0): # if everything is added to two lists
            return news_headers_per_page, datetime_per_page, contents_per_page

## Functions for Scraping Multiple Pages

In [6]:
# function to get next page url
def navigate_next_page(soup):
    next_page_soup = soup.find("a", {"aria-label":"next-page", "href":True})
    new_url = next_page_soup.attrs['href']
    return new_url

In [7]:
# function to strip away unnecessary characters in a string
def remove_chars(original_str, list_of_unwanted_chars):
    new_str = ""
    for digit in original_str:
        if digit in list_of_unwanted_chars:
            continue
        new_str += digit
    return new_str

In [8]:
# function to get the last page index
def get_max_page(soup):
    last_page_soup = soup.find("a", {"class":"last", "href":True})
    last_page_index_string = last_page_soup.string.strip() # get something like 1,000
    last_page_index_string = remove_chars(last_page_index_string, [',', '.', ';']) # strip away unwanted commas
    last_page_index = int(last_page_index_string)
    return last_page_index

## Function for Excel

In [9]:
# function to produce a spreadsheet
def export_excel(first_list, second_list, third_list):
    KT = {}
    KT['News Header'] = first_list
    KT['Time'] = second_list
    KT['Content'] = third_list
    df = pd.DataFrame({key:pd.Series(value) for key, value in KT.items()})
    df.to_excel('KT_webscraped.xlsx', index=False)
    return df

## Scraping Everything to Spreadsheet

In [10]:
# the main function
def main(web_url):
    news_headers = []
    datetime = []
    contents = []

    # for the first page
    complete_url = web_url # initial url
    soup = web_scraper(complete_url) # get soup
    last_page_index = get_max_page(soup) # get last page index
    news_headers_soup, datetime_soup = soup_parser(soup) # get specific elements in a soup
    print("Total pages:", last_page_index)
    
    # extract data
    news_headers_per_page, datetime_per_page, contents_per_page = list_append_per_page(news_headers_soup, datetime_soup)

    # append data to lists
    news_headers += news_headers_per_page
    datetime += datetime_per_page
    contents += contents_per_page

    '''NOTE: There are too many pages so it is not efficient to uncomment this without the need.'''
    # from the second page to the last page # insert last_page_index to scrape everything
    for i in range(1): # scrape only the second page as an example
        try:
            print(complete_url)
            complete_url = navigate_next_page(soup) # get next url
            soup = web_scraper(complete_url) # get soup
            news_headers_soup, datetime_soup = soup_parser(soup) # get specific elements in a soup

            # extract data
            news_headers_per_page, datetime_per_page, contents_per_page = list_append_per_page(news_headers_soup, datetime_soup)
            
            # append data to lists
            news_headers += news_headers_per_page
            datetime += datetime_per_page
            contents += contents_per_page
        
        except AttributeError: # scraping the page after the last page will cause error
            print("The end of the pages is reached.")

    return export_excel(news_headers, datetime, contents) # export the spreadsheet # return df

In [11]:
link = "https://yktnews.com/category/news/"
df = main(link)

Total pages: 1585
https://yktnews.com/category/news/


In [12]:
df

,News Header,Time,Content
0,စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြိ...,"July 24, 2024",စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြိ...
1,ဆားလင်းကြီးမြို့နယ်ရှိ စစ်ရှောင် ပြည်သူများကို...,"July 24, 2024",ဆားလင်းကြီးမြို့နယ်ရှိ စစ်ရှောင် ပြည်သူများကို...
2,ပခုက္ကူမြို့တွင် KTV မှ ပြန်လာသော ဗိုလ်ကြီး ၁ ...,"July 24, 2024",ပခုက္ကူမြို့တွင် မှ ပြန်လာသော ဗိုလ်ကြီး ၁ ဦးက...
3,ထိုင်းနိုင်ငံ စမူပါကန် ဘောလုံးကွင်းတွင် နဂါးဖြ...,"July 24, 2024",ထိုင်းနိုင်ငံ စမူပါကန် ဘောလုံးကွင်းတွင် နဂါးဖြ...
4,စစ်ကောင်စီနှင့် လက်အောက်ခံတပ်ဖွဲ့များက ၃ နှစ်က...,"July 24, 2024",ရန်ကုန်| ဇူလိုင် ၂၄မြန်မာနိုင်ငံတွင် ၂၀၂၁ ခုနှ...
5,အကြမ်းဖက်စစ်ကောင်စီက ထင်ရှား လူသိများသော စီးပွ...,"July 24, 2024",ရန်ကုန်၊ ဇူလိုင် ၂၄အကြမ်းဖက်စစ်ကောင်စီက ထင်ရှာ...
6,မလေးရှားနိုင်ငံသို့ တရားမဝင် သွားရောက် အလုပ်လု...,"July 24, 2024",ရန်ကုန်၊ ဇူလိုင် ၂၄မလေးရှားနိုင်ငံသို့ သွားရော...
7,စဉ့်ကူးမြို့နယ်၊ ကျည်တောက်ပေါက်ရွာကို စစ်တပ်က ...,"July 24, 2024",စဉ့်ကူးမြို့နယ်၊ ကျည်တောက်ပေါက်ရွာကို စစ်တပ်က ...
8,ဆားလင်းကြီးမြို့နယ်အတွင်းရှိ ရေဘေးသင့်ပြည်သူမျ...,"July 24, 2024",ဆားလင်းကြီးမြို့နယ်အတွင်းရှိ ရေဘေးသင့်ပြည်သူမျ...
9,အမေရိကန်တစ်ဒေါ်လာ ငွေကျပ် ၅၀၀၀ ဝန်းကျင်သို့ ပြ...,"July 24, 2024",ရန်ကုန်၊ ဇူလိုင် ၂၄မြန်မာနိုင်ငံ၏ ပြည်တွင်းစီး...


## Duplicates?!??
The DataFrame contains duplicate entries. To eradicate this, we can update our `soup_parser()` function. To do that, we have to experiment and inspect elements.

In [13]:
# what does old soup_parser produces?
s = web_scraper(link)
old_parser_result = soup_parser(s)
len(old_parser_result) # check total indexes

2

In [14]:
# check elements
print(old_parser_result[0][0])
print(old_parser_result[0][-1])

<a href="https://yktnews.com/2024/07/175459/" rel="bookmark" title="စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြို့ကို တတိယအကြိမ်မြောက် လေယာဉ်ဖြင့် ဗုံးကျဲမှု အရပ်သား ၂ ဦး ထိခိုက်ဒဏ်ရာရ">စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြို့ကို တတိယအကြိမ်မြောက် လေယာဉ်ဖြင့် ဗုံးကျဲမှု အရပ်သား ၂ ဦး ထိခိုက်ဒဏ်ရာရ</a>
<a href="https://yktnews.com/2024/07/175464/" rel="bookmark" title="မိုးမိတ်မြို့ရှိ စစ်တပ် ဗျူဟာကုန်းကို TNLA က တိုက်ခိုက်နေပြီး နောင်ချိုမြို့နယ်တွင်လည်း တိုက်ပွဲများ ဆက်လက် ပြင်းထန်နေ">မိုးမိတ်မြို့ရှိ စစ်တပ် ဗျူဟာကုန်းကို TNLA က တိုက်ခိုက်နေပြီး နောင်ချိုမြို့နယ်တွင်လည်း တိုက်ပွဲများ ဆက်လက် ပြင်းထန်နေ</a>


We can see that there is no significant difference. Let's inspect their parents.

In [15]:
# check elements' parents
print(old_parser_result[0][0].parent)
print(old_parser_result[0][-1].parent)

<p class="entry-title td-module-title"><a href="https://yktnews.com/2024/07/175459/" rel="bookmark" title="စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြို့ကို တတိယအကြိမ်မြောက် လေယာဉ်ဖြင့် ဗုံးကျဲမှု အရပ်သား ၂ ဦး ထိခိုက်ဒဏ်ရာရ">စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြို့ကို တတိယအကြိမ်မြောက် လေယာဉ်ဖြင့် ဗုံးကျဲမှု အရပ်သား ၂ ဦး ထိခိုက်ဒဏ်ရာရ</a></p>
<h3 class="entry-title td-module-title"><a href="https://yktnews.com/2024/07/175464/" rel="bookmark" title="မိုးမိတ်မြို့ရှိ စစ်တပ် ဗျူဟာကုန်းကို TNLA က တိုက်ခိုက်နေပြီး နောင်ချိုမြို့နယ်တွင်လည်း တိုက်ပွဲများ ဆက်လက် ပြင်းထန်နေ">မိုးမိတ်မြို့ရှိ စစ်တပ် ဗျူဟာကုန်းကို TNLA က တိုက်ခိုက်နေပြီး နောင်ချိုမြို့နယ်တွင်လည်း တိုက်ပွဲများ ဆက်လက် ပြင်းထန်နေ</a></h3>


### 1. First Insight into <i>soup_parser()</i>
Now, we can see that they are of difference HTML tags. If we inspect the original website, we would find that our news are written in `<p>` tags. Other tags are used for recommended news. So, we will choose only `<p>` elements for our `news headers`.

### 2. Second Insight into <i>soup_parser()</i>
Recall that `datetime` is mapped to the titles (news headers). If we remove certain elements from `news headers`, we must also delete the corresponding entries.

In [16]:
# check elements' time
print(old_parser_result[1][0])
print(old_parser_result[1][-1])

<time class="entry-date updated td-module-date" datetime="2024-07-24T20:57:17+07:00">July 24, 2024</time>
<time class="entry-date updated td-module-date" datetime="2024-07-24T20:59:41+07:00">July 24, 2024</time>


In [17]:
# check elements' time's parents
print(old_parser_result[1][0].parent)
print(old_parser_result[1][-1].parent)

<span class="td-post-date"><time class="entry-date updated td-module-date" datetime="2024-07-24T20:57:17+07:00">July 24, 2024</time></span>
<span class="td-post-date"><time class="entry-date updated td-module-date" datetime="2024-07-24T20:59:41+07:00">July 24, 2024</time></span>


We cannot easily spot their differences like we did it in above. So, we keep searching a thing to distinguish.

In [18]:
# check its parent
print(old_parser_result[1][0].parent.parent)
print(old_parser_result[1][-1].parent.parent)

<span class="td-author-date">
<span class="td-post-author-name"><a href="https://yktnews.com/author/phyowaisoeyktnews-com/">Yangon Khit Thit News Agency</a> <span>-</span> </span>                                    <span class="td-post-date"><time class="entry-date updated td-module-date" datetime="2024-07-24T20:57:17+07:00">July 24, 2024</time></span>                                                                                                        </span>
<span class="td-author-date">
<span class="td-post-date"><time class="entry-date updated td-module-date" datetime="2024-07-24T20:59:41+07:00">July 24, 2024</time></span>                                                                                                        </span>


In [19]:
# check its parent
print(old_parser_result[1][0].parent.parent.parent)
print(old_parser_result[1][-1].parent.parent.parent)

<div class="td-editor-date">
<span class="td-author-date">
<span class="td-post-author-name"><a href="https://yktnews.com/author/phyowaisoeyktnews-com/">Yangon Khit Thit News Agency</a> <span>-</span> </span>                                    <span class="td-post-date"><time class="entry-date updated td-module-date" datetime="2024-07-24T20:57:17+07:00">July 24, 2024</time></span>                                                                                                        </span>
</div>
<div class="td-editor-date">
<span class="td-author-date">
<span class="td-post-date"><time class="entry-date updated td-module-date" datetime="2024-07-24T20:59:41+07:00">July 24, 2024</time></span>                                                                                                        </span>
</div>


In [20]:
# check its parent
print(old_parser_result[1][0].parent.parent.parent.parent)
print(old_parser_result[1][-1].parent.parent.parent.parent)

<div class="td-module-meta-info">
<p class="entry-title td-module-title"><a href="https://yktnews.com/2024/07/175459/" rel="bookmark" title="စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြို့ကို တတိယအကြိမ်မြောက် လေယာဉ်ဖြင့် ဗုံးကျဲမှု အရပ်သား ၂ ဦး ထိခိုက်ဒဏ်ရာရ">စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြို့ကို တတိယအကြိမ်မြောက် လေယာဉ်ဖြင့် ဗုံးကျဲမှု အရပ်သား ၂ ဦး ထိခိုက်ဒဏ်ရာရ</a></p>
<div class="td-editor-date">
<span class="td-author-date">
<span class="td-post-author-name"><a href="https://yktnews.com/author/phyowaisoeyktnews-com/">Yangon Khit Thit News Agency</a> <span>-</span> </span>                                    <span class="td-post-date"><time class="entry-date updated td-module-date" datetime="2024-07-24T20:57:17+07:00">July 24, 2024</time></span>                                                                                                        </span>
</div>
<div class="td-excerpt">
စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြို့ကို တတိယအကြိမ်မြောက် လေယာဉ်ဖြင့် ဗုံး

### 3. Third Insight into <i>soup_parser()</i>
Now that we can highlight a unique distinguishment (`<p>` and `<h3>` in this case), we are going to use it to remove corresponding entries of `datetime`.

In [21]:
# get a property to make conditionals
print(old_parser_result[1][0].parent.parent.parent.parent.contents[1].name)
print(old_parser_result[1][-1].parent.parent.parent.parent.contents[1].name)

p
h3


## Scraping Everything with Duplicates Filtered

In [22]:
# redefine function to get specific elements within a soup
def soup_parser(soup):
    news_headers_soup = soup.find_all("a", {"class":None, "rel":"bookmark"})
    datetime_soup = soup.find_all("time")
    
    '''The soup contain elements of the Editor Picks and the Latest News which are redundant information, sometimes making duplicate entries.'''
    # removing specific elements mentioned above
    for a_element in news_headers_soup:
        if a_element.parent.name == 'p': # check its parent # 'p' is desired # 'h3' for example is not desired
            continue
        news_headers_soup.remove(a_element) # remove unwanted entries
    for t_element in datetime_soup:
        if t_element.parent.parent.parent.parent.contents[1].name == 'p':
            continue
        datetime_soup.remove(t_element)
    # uncomment above code block if you want to scrape everything

    if len(news_headers_soup) != len(datetime_soup):
        raise Exception("Failed to map Titles to Dates.") # debugging line
    return news_headers_soup, datetime_soup

In [23]:
link = "https://yktnews.com/category/news/"
df = main(link)

Total pages: 1585
https://yktnews.com/category/news/


In [24]:
df

,News Header,Time,Content
0,ပခုက္ကူမြို့တွင် KTV မှ ပြန်လာသော ဗိုလ်ကြီး ၁ ...,"July 24, 2024",ပခုက္ကူမြို့တွင် မှ ပြန်လာသော ဗိုလ်ကြီး ၁ ဦးက...
1,ထိုင်းနိုင်ငံ စမူပါကန် ဘောလုံးကွင်းတွင် နဂါးဖြ...,"July 24, 2024",ထိုင်းနိုင်ငံ စမူပါကန် ဘောလုံးကွင်းတွင် နဂါးဖြ...
2,စစ်ကောင်စီနှင့် လက်အောက်ခံတပ်ဖွဲ့များက ၃ နှစ်က...,"July 24, 2024",ရန်ကုန်| ဇူလိုင် ၂၄မြန်မာနိုင်ငံတွင် ၂၀၂၁ ခုနှ...
3,အကြမ်းဖက်စစ်ကောင်စီက ထင်ရှား လူသိများသော စီးပွ...,"July 24, 2024",ရန်ကုန်၊ ဇူလိုင် ၂၄အကြမ်းဖက်စစ်ကောင်စီက ထင်ရှာ...
4,မလေးရှားနိုင်ငံသို့ တရားမဝင် သွားရောက် အလုပ်လု...,"July 24, 2024",ရန်ကုန်၊ ဇူလိုင် ၂၄မလေးရှားနိုင်ငံသို့ သွားရော...
5,စဉ့်ကူးမြို့နယ်၊ ကျည်တောက်ပေါက်ရွာကို စစ်တပ်က ...,"July 24, 2024",စဉ့်ကူးမြို့နယ်၊ ကျည်တောက်ပေါက်ရွာကို စစ်တပ်က ...
6,ဆားလင်းကြီးမြို့နယ်အတွင်းရှိ ရေဘေးသင့်ပြည်သူမျ...,"July 24, 2024",ဆားလင်းကြီးမြို့နယ်အတွင်းရှိ ရေဘေးသင့်ပြည်သူမျ...
7,အမေရိကန်တစ်ဒေါ်လာ ငွေကျပ် ၅၀၀၀ ဝန်းကျင်သို့ ပြ...,"July 24, 2024",ရန်ကုန်၊ ဇူလိုင် ၂၄မြန်မာနိုင်ငံ၏ ပြည်တွင်းစီး...
8,စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြိ...,"July 24, 2024",စစ်ကောင်စီက ယနေ့ နံနက်အစောပိုင်း လောက်ကိုင်မြိ...
9,အောင်လံလွှင့်မည် ဒေါင်းသွေးနီ သပိတ်စစ်ကြောင်းက...,"July 24, 2024",အောင်လံလွှင့်မည် ဒေါင်းသွေးနီ သပိတ်စစ်ကြောင်းက...


## Conclusion
I find that Yangon Khit Thit (YKT) suggests certain news by the editor, unlike BBC Burmese. These kinds of news are already included in pages, causing duplicates. Furthermore, YKT uses commas(,) in page numbers. Inserting specific solutions to such differences is required if I were to use the BBC Burmese Webscraper script.